In [2]:
from torch.utils.data import Dataset
import torch
import glob
import pandas as pd
from datetime import datetime
import os
import math
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import random

from scipy import signal

/home/yhbedoya/VirtualEnvs/SHMmae_venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
start_time, end_time = "05/12/2021 06:00", "05/12/2021 07:00"
path = '/home/yhbedoya/Repositories/SHM-MAE/traffic/20211205/'

print(f'reading CSV files')
start = datetime.strptime(start_time, '%d/%m/%Y %H:%M')
end = datetime.strptime(end_time, '%d/%m/%Y %H:%M')

ldf = list()
for p in tqdm(glob.glob(path + "*.csv")):
    name = os.path.split(p)[-1]
    nstr = datetime.strptime(name, 'traffic_%Y%m%dH%H%M%S.csv')
    if start <= nstr < end:
        df_tmp = pd.read_csv(p)
        c_drop = set(df_tmp.columns) - set(["sens_pos", "z", "ts"])
        if len(c_drop) > 0:
            df_tmp.drop(columns=list(c_drop), inplace=True)
        ldf.append(df_tmp)
df = pd.concat(ldf).sort_values(by=['sens_pos', 'ts'])
df.reset_index(inplace=True, drop=True)

#df = df[df['sens_pos'].isin(self.sensors)]
df['ts'] = pd.to_datetime(df['ts'], unit='ms')
df["zN"] = df["z"]-np.mean(df["z"])
df["vars"] = df["zN"].rolling(window=10).var().fillna(0)

reading CSV files


100%|██████████| 1450/1450 [00:28<00:00, 50.79it/s]


In [4]:
distanceToSensor = {}
with open('/home/yhbedoya/Repositories/SHM-MAE/LabelGeneration/distanceToSensor.csv') as f:
    for line in f.readlines():
        sensor, distance = line.replace("'", "").replace("\n","").split(",")
        distanceToSensor[sensor] = float(distance)
distanceToSensor

{'C1.1.1': 600.0,
 'C1.1.2': 601.37,
 'C1.1.3': 603.42,
 'C2.1.2': 624.72,
 'C2.1.3': 628.07,
 'C2.1.4': 633.07,
 'C3.1.2': 656.98,
 'C3.1.3': 660.33,
 'C3.1.4': 665.33,
 'C4.1.2': 690.97,
 'C4.1.3': 694.32,
 'C4.1.4': 699.32,
 'C5.1.2': 724.98,
 'C5.1.3': 728.33,
 'C5.1.4': 733.33,
 'C6.1.2': 758.99,
 'C6.1.3': 762.34,
 'C6.1.4': 767.34,
 'C7.1.2': 791.25,
 'C7.1.3': 794.6,
 'C7.1.4': 799.6,
 'C8.1.2': 823.25,
 'C8.1.3': 826.6,
 'C8.1.4': 831.6,
 'C9.1.2': 855.29,
 'C9.1.3': 858.64,
 'C9.1.4': 863.64,
 'C10.1.2': 887.02,
 'C10.1.3': 892.67,
 'C10.1.4': 899.42,
 'C11.1.2': 929.96,
 'C11.1.3': 933.31,
 'C11.1.4': 938.31,
 'C12.1.2': 965.12,
 'C12.1.3': 968.47,
 'C12.1.4': 973.47,
 'C13.1.2': 1000.32,
 'C13.1.3': 1003.67,
 'C13.1.4': 1008.67,
 'C14.1.2': 1035.53,
 'C14.1.3': 1038.88,
 'C14.1.4': 1043.88,
 'C15.1.2': 1070.67,
 'C15.1.3': 1074.02,
 'C15.1.4': 1079.02,
 'C16.1.2': 1108.99,
 'C16.1.3': 1112.34,
 'C16.1.4': 1117.34,
 'C17.1.2': 1147.31,
 'C17.1.3': 1150.66,
 'C17.1.4': 1155.6

In [5]:
distanceToSensor["C8.2.3"]

826.6

In [4]:
pesaDataDf = pd.read_csv("/home/yhbedoya/Repositories/SHM-MAE/dati_pese_dinamiche/dati 2021-12-04_2021-12-12 pesa km 104,450.csv", sep=";", index_col=0)
pesaDataDf = pesaDataDf[["Id", "StartTimeStr", "ClassId", "GrossWeight", "Velocity", "VelocityUnit"]]
pesaDataDf["Time"] = pd.to_datetime(pesaDataDf["StartTimeStr"])
pesaDataDf["Time"] = pesaDataDf["Time"].dt.strftime('%Y-%d-%m %H:%M:00')
pesaDataDf["Time"] = pd.to_datetime(pesaDataDf["Time"]) + pd.to_timedelta(-1,'H')
#pesaDataDf["Time"] = pd.to_datetime(pesaDataDf["Time"]).dt.strftime('%Y-%m-%d %H:%M:00')
pesaDataDf.sort_values(by="Id", inplace=True)
pesaDataDf = pesaDataDf[(pesaDataDf["Time"]>="2021-12-05 06:00:00") & (pesaDataDf["Time"]<="2021-12-05 06:59:00")]
pesaDataDf.reset_index(drop=True, inplace=True)

In [5]:
threshold = 1e-6
minDuration = 0.25
def groupsGenerator(data, minTime, maxTime):
    slice = data[(data["ts"]>= minTime) & (data["ts"]<= maxTime)]
    slice["outlier"] = slice["vars"].apply(lambda x: x>=threshold)
    outliers = slice[slice["outlier"] == True].reset_index().to_dict("records")

    if len(outliers) == 0:
        return None

    last = minTime
    timeStart = outliers[0]["ts"]
    flag = True
    groups = []
    groupTimes = []
    groupIndexes = []
    groupVars = []
    label = np.nan
    groupId = 0
    for outlier in outliers:
        if ((outlier["ts"] - last).total_seconds() < 2) or flag:
            groupTimes.append(outlier["ts"])
            groupVars.append(outlier["vars"])
            flag = False
            timeEnd = outlier["ts"]
        else:
            start, end = min(groupTimes), max(groupTimes)
            groupSignal = data[(data["ts"]>= start) & (data["ts"]<= end)]["zN"]
            signalPower = np.sqrt(np.mean(np.array(groupSignal)**2))**2 
            pointMaxVar = groupTimes[np.argmax(groupVars)]
            if ((end - start).total_seconds() > minDuration):
                label = {"groupId": groupId,"start": start, "end": end, "signalPower": signalPower, 
                "pointMaxVar": pointMaxVar}
                groups.append(label)
            groupId += 1
            groupTimes = [outlier["ts"],]
            groupVars = [outlier["vars"],]
        last = outlier["ts"]

    start, end = min(groupTimes), max(groupTimes)
    groupSignal = data[(data["ts"]>= start) & (data["ts"]<= end)]["zN"]
    signalPower = np.sqrt(np.mean(np.array(groupSignal)**2))**2 
    pointMaxVar = groupTimes[np.argmax(groupVars)]
    if ((end-start).total_seconds() > minDuration):
            label = {"groupId": groupId,"start": start, "end": end, "signalPower": signalPower, 
            "pointMaxVar": pointMaxVar}
            groups.append(label)

    if len(groups)>0:
        groupsDf = pd.DataFrame(groups).sort_values("signalPower", ascending=False)
    else:
        groupsDf = pd.DataFrame()

    return groupsDf

In [6]:
def extract_numbers(string):
    first, second, third = string.split('.')
    return int(first[1:]), int(third)

In [7]:
sensorsList = df["sens_pos"].unique()
sensorsList = sorted(sensorsList, key=extract_numbers)

In [9]:
precSensors ={}

for sensor in sensorsList:
    if sensor == 'C1.1.1' or sensor == 'C1.2.1':
        continue
    line = int(sensor.split(".")[1])
    distanceToScale = distanceToSensor[sensor]
    sensors = []
    distances = []
    for k, v in distanceToSensor.items():
        sensors.append(k)
        distances.append(distanceToScale-v)
    distancesDf = pd.DataFrame({
        "sensor": sensors,
        "distance": distances
    })

    predSensor = distancesDf[distancesDf["distance"]>0].sort_values("distance")
    predSensor = predSensor[predSensor["sensor"].apply(lambda x: int(x.split(".")[1]) == line)].iloc[0]

    precSensors[sensor] = {"sensor": predSensor["sensor"],
                            "distance": predSensor["distance"]}

precSensors['C1.1.1'] = {"sensor": "Scale",
                        "distance": 600}

precSensors['C1.2.1'] = {"sensor": "Scale",
                        "distance": 600}


In [19]:
sensorsList[30:]

['C6.1.2',
 'C6.2.2',
 'C6.1.3',
 'C6.2.3',
 'C6.1.4',
 'C6.2.4',
 'C7.1.2',
 'C7.2.2',
 'C7.1.3',
 'C7.2.3',
 'C7.1.4',
 'C7.2.4',
 'C8.1.2',
 'C8.2.2',
 'C8.1.3',
 'C8.1.4',
 'C8.2.4',
 'C9.1.2',
 'C9.2.2',
 'C9.1.3',
 'C9.1.4',
 'C9.2.4',
 'C10.1.2',
 'C10.2.2',
 'C10.1.3',
 'C10.2.3',
 'C10.1.4',
 'C10.2.4',
 'C11.1.2',
 'C11.2.2',
 'C11.1.3',
 'C11.2.3',
 'C11.1.4',
 'C11.2.4',
 'C12.1.2',
 'C12.2.2',
 'C12.1.3',
 'C12.2.3',
 'C12.1.4',
 'C12.2.4',
 'C13.1.2',
 'C13.2.2',
 'C13.1.3',
 'C13.2.3',
 'C13.1.4',
 'C13.2.4',
 'C14.1.2',
 'C14.2.2',
 'C14.1.3',
 'C14.2.3',
 'C14.1.4',
 'C14.2.4',
 'C15.1.2',
 'C15.2.2',
 'C15.1.3',
 'C15.2.3',
 'C15.1.4',
 'C15.2.4',
 'C16.1.2',
 'C16.2.2',
 'C16.1.3',
 'C16.2.3',
 'C16.1.4',
 'C16.2.4',
 'C17.1.2',
 'C17.2.2',
 'C17.1.3',
 'C17.1.4',
 'C17.2.4',
 'C18.1.2',
 'C18.2.2',
 'C18.1.3',
 'C18.2.3',
 'C18.1.4',
 'C18.2.4']

In [11]:
sensorLabelsDfDict = {}
def timesCalc(row):
    predSensor = precSensors[sensor]["sensor"]
    predDistance = precSensors[sensor]["distance"]
    if predSensor == "Scale":
        precLabelDf = np.nan
        precLabel = np.nan
        flag = False
    else:
        precLabelDf = sensorLabelsDfDict[predSensor]
        precLabelRow = precLabelDf[precLabelDf["Id"] == row["Id"]]
        precLabel = precLabelRow["labels"]
        if isinstance(precLabel, dict):
            flag = True
        else:
            flag = False

    if flag:
        delta = (float(predDistance)/(row["Velocity"]/3.6))
        row["EstimatedTime"] = pd.to_datetime(precLabelRow["label"]["pointMaxVar"]) + pd.to_timedelta(delta*0.8,'S')
        row["MaxTime"] = pd.to_datetime(precLabelRow["label"]["pointMaxVar"]) + pd.to_timedelta(delta*1.2,'S')
    else:
        delta = distanceToSensor[sensor]/(row["Velocity"]/3.6)
        row["EstimatedTime"] = pd.to_datetime(row["Time"]) + pd.to_timedelta(delta*0.8,'S')
        row["MaxTime"] = pd.to_datetime(row["Time"]) + pd.to_timedelta(delta*1.2,'S')

    return row

In [13]:
groupsDfList = []

for sensor in tqdm(sensorsList):
    assignedLabels = {}
    assignedLabels2 = {}
    sensorLabelsDf = pesaDataDf.copy(deep=True)
    sensorLabelsDf = sensorLabelsDf.apply(timesCalc, axis=1)

    minTime = sensorLabelsDf["EstimatedTime"].min()
    maxTime = sensorLabelsDf["MaxTime"].max()
    sensorLabelsDf.sort_values("GrossWeight", inplace=True, ascending=False)

    sensorData = df[df["sens_pos"]==sensor]
    groupsDf = groupsGenerator(sensorData, minTime, maxTime)
    availableGroupsDf = groupsDf.copy(deep=True)
    if availableGroupsDf.empty:
        continue
    for index, row in sensorLabelsDf.iterrows():
        if row["Id"] in assignedLabels:
            continue
        
        if availableGroupsDf.empty:
            break

        candidatesDf = availableGroupsDf[(row["EstimatedTime"] <= availableGroupsDf["start"]) & (availableGroupsDf["start"] <= row["MaxTime"])]
        if not candidatesDf.empty:
            assignedLabels[row["Id"]] = candidatesDf.iloc[0].to_dict()
            assignedLabels2[candidatesDf.iloc[0]["groupId"]] = row["Id"]
            availableGroupsDf.drop(candidatesDf.index[0], inplace=True)
    
    sensorLabelsDf["sens_pos"] = sensor
    sensorLabelsDf["labels"] = sensorLabelsDf.apply(lambda row: assignedLabels[row["Id"]] if row["Id"] in assignedLabels else np.nan, axis=1)
    sensorLabelsDf.sort_values("Id", inplace=True)
    groupsDf["sens_pos"] = sensor
    groupsDf["labels"] = groupsDf.apply(lambda row: assignedLabels2[row["groupId"]] if row["groupId"] in assignedLabels2 else np.nan, axis=1)
    groupsDf.sort_values("groupId", inplace=True)
    groupsDf.dropna(inplace=True)

    sensorLabelsDfDict[sensor] = sensorLabelsDf
    groupsDfList.append(groupsDf)


  0%|          | 0/105 [00:00<?, ?it/s]/home/yhbedoya/VirtualEnvs/SHMmae_venv/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
  1%|          | 1/105 [00:02<03:33,  2.05s/it]/home/yhbedoya/VirtualEnvs/SHMmae_venv/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
  2%|▏         | 2/105 [00:03<02:54,  1.70s/it]/home/yhbedoya/VirtualEnvs/SHMmae_venv/lib/python3.7/site-packages/ipykernel_launcher.py:5: Set

KeyError: 'C8.2.3'

In [ ]:
labelsDf = pd.concat(sensorLabelsDfList)
groupsDf =  pd.concat(groupsDfList)

In [7]:
print(f"Total labels: {len(labelsDf)}")
totnan = labelsDf["labels"].isna().sum()
print(f"Total nan labels: {totnan}")
print(f"Proportion of match labels: {1-(totnan/len(labelsDf))}")

Total labels: 14805
Total nan labels: 9338
Proportion of match labels: 0.3692671394799054


In [24]:
labelsDf

NameError: name 'labelsDf' is not defined

In [8]:
groupsDf

,groupId,start,end,signalPower,pointMaxVar,sens_pos,labels
0,1,2021-12-05 00:03:59.811,2021-12-05 00:04:09.841,0.000380,2021-12-05 00:04:00.481,C1.1.1,52373.0
1,7,2021-12-05 00:10:07.231,2021-12-05 00:10:16.101,0.000382,2021-12-05 00:10:08.211,C1.1.1,52380.0
2,9,2021-12-05 00:13:37.161,2021-12-05 00:13:37.671,0.000385,2021-12-05 00:13:37.161,C1.1.1,52382.0
5,16,2021-12-05 00:23:15.196,2021-12-05 00:23:15.726,0.000369,2021-12-05 00:23:15.226,C1.1.1,52383.0
6,18,2021-12-05 00:24:58.476,2021-12-05 00:25:05.636,0.000383,2021-12-05 00:24:59.446,C1.1.1,52384.0
...,...,...,...,...,...,...,...
8,15,2021-12-05 00:29:41.060,2021-12-05 00:29:41.720,0.000006,2021-12-05 00:29:41.600,C9.2.4,52386.0
9,17,2021-12-05 00:49:16.708,2021-12-05 00:50:01.428,0.000009,2021-12-05 00:49:27.218,C9.2.4,52389.0
10,18,2021-12-05 00:50:07.348,2021-12-05 00:50:07.938,0.000006,2021-12-05 00:50:07.388,C9.2.4,52390.0
11,20,2021-12-05 00:52:22.448,2021-12-05 00:52:23.038,0.000006,2021-12-05 00:52:22.948,C9.2.4,52393.0


In [9]:
counts = groupsDf.groupby('sens_pos').count()
counts.sort_values("labels", ascending=False, inplace=True)
counts

,groupId,start,end,signalPower,pointMaxVar,labels
sens_pos,,,,,,
C12.1.3,109,109,109,109,109,109
C1.1.2,108,108,108,108,108,108
C1.1.3,103,103,103,103,103,103
C1.2.3,103,103,103,103,103,103
C9.1.3,92,92,92,92,92,92
...,...,...,...,...,...,...
C2.1.4,20,20,20,20,20,20
C10.2.2,12,12,12,12,12,12
C10.2.3,7,7,7,7,7,7


### Assign label to windows

In [9]:
sampleRate = 100
frameLength = 256
stepLength = 64
windowLength= 6000
windowStep = 1500

In [10]:
def _transformation(slice):
    sliceN = slice-torch.mean(slice)
    frequencies, times, spectrogram = signal.spectrogram(sliceN,sampleRate,nfft=frameLength,noverlap=(frameLength - stepLength), nperseg=frameLength,mode='psd')

    return frequencies, times, np.log10(spectrogram)

def _normalizer( spectrogram):
    spectrogramNorm = (spectrogram - min) / (max - min)
    return spectrogramNorm

def butter_bandpass( lowcut, highcut, fs, order=5):
    return signal.butter(order, [1, 49], fs=fs, btype='band')

def butter_bandpass_filter( slice, lowcut, highcut, fs, order=5):
    sliceN = slice-np.mean(np.array(slice))
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = signal.lfilter(b, a, sliceN)
    return y

def power( slice):
    signalPower = np.sqrt(np.mean(np.array(slice)**2))**2
    return signalPower

In [11]:
def _labelAssigner(timeSlice, sensor):
    start, end = timeSlice.min(), timeSlice.max()
    vehiclesInSliceDf = groupsDf[(groupsDf["pointMaxVar"]>=start) &
    (groupsDf["pointMaxVar"]<=end) &
    (groupsDf["sens_pos"]==sensor)]
    return vehiclesInSliceDf

In [12]:
def _partitioner(data):
    sensors = data['sens_pos'].unique().tolist()
    print(f'start partitioner')
    partitions = {}
    cumulatedWindows = 0
    limits = dict()
    print(f'Generating windows')
    for sensor in tqdm(sensors):
        sensorData = data[data['sens_pos']==sensor]
        totalFrames = sensorData.shape[0]
        totalWindows = math.ceil((totalFrames-windowLength)/windowStep)
        start = cumulatedWindows
        cumulatedWindows += totalWindows
        end = cumulatedWindows
        indexStart = sensorData.index[0]
        partitions[sensor]= (start, end, indexStart)


    timeData = torch.tensor(data["z"].values, dtype=torch.float64)
    timestamps = data["ts"]
    cummulator = -1

    mins = list()
    maxs = list()
    print(f'Defining useful windows limits')
    indexes = list(range(0, cumulatedWindows))
    random.shuffle(indexes)

    for index in tqdm(indexes):
        if cummulator >= 500000:
            break
        for sensor,v in partitions.items():
            if index in range(v[0], v[1]):
                start = v[2]+(index-v[0])*windowStep
                timeSlice = timestamps[start: start+windowLength]
                label = _labelAssigner(timeSlice, sensor)
                filteredSlice = butter_bandpass_filter(timeData[start: start+windowLength], 0, 50, sampleRate)
                signalPower = power(filteredSlice)

                if (signalPower>1.25*10**-6) or (len(label)>0):
                    cummulator += 1
                    limits[cummulator] = (start, start+windowLength, label, timeSlice)
                    slice = timeData[start:start+windowLength]
                    frequencies, times, spectrogram = _transformation(torch.tensor(slice, dtype=torch.float64))
                    mins.append(np.min(np.array(spectrogram)))
                    maxs.append(np.max(np.array(spectrogram)))
                break
    print(f'Total windows in dataset: {cummulator}')
    min = np.min(np.array(mins))
    max = np.max(np.array(maxs))   
    print(f'General min: {min}')
    print(f'General max: {max}')
    return timeData, limits, cummulator, min, max

In [13]:
newDataDf = df.copy(deep=True)
timeData, limits, cummulator, min, max = _partitioner(newDataDf)

start partitioner
Generating windows


100%|██████████| 105/105 [03:00<00:00,  1.72s/it]


Defining useful windows limits


100%|██████████| 24780/24780 [00:38<00:00, 652.05it/s]

Total windows in dataset: 3842
General min: -21.0291564449071
General max: -3.5637023861028587


In [17]:
def __getitem__(index):
    start, end, label, timeSlice = limits[index]
    slice = df[start:end]
    z, vars =  torch.tensor(slice["z"].tolist(), dtype=torch.float64), slice["vars"]
    frequencies, times, spectrogram = _transformation(z)
    spectrogram = torch.unsqueeze(torch.tensor(spectrogram, dtype=torch.float64), 0)
    NormSpect = _normalizer(spectrogram).type(torch.float16)
    #print(f'type {type(NormSpect)}, inp shape: {slice.shape} out shape: {NormSpect.shape}')
    return frequencies, times, spectrogram, z, label, vars, timeSlice

In [18]:
def plotSpect(frequencies, times, spectrogram, label):
    plt.pcolormesh(times, frequencies, 10*(np.squeeze(spectrogram)), vmin=-150, vmax=-50)
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.colorbar(format="%+2.f", label='dB')

In [19]:
def multiplot(frequencies, times, spectrogram, slice, label, vars, timeSlice):
    #print(f"{np.min(timeSlice)}, {np.max(timeSlice)}")
    provDf = pd.DataFrame({"z":slice, "ts":timeSlice}).reset_index(drop=True)
    lines = []
    for index, row in label.iterrows(): 
        subSetDf = provDf[(provDf["ts"]>=row["start"])&(provDf["ts"]<=row["end"])]
        subSetDf2 = provDf[provDf["ts"]<=row["pointMaxVar"]]
        minLine, maxLine, labelLine = np.min(subSetDf.index), np.max(subSetDf.index), np.max(subSetDf2.index)
        lines.append((minLine, maxLine, labelLine))
    
    plt.rcParams['figure.figsize'] = [30, 8]

    plt.subplot(1, 3, 1)
    plotSpect(frequencies, times, spectrogram, "original")

    plt.subplot(1, 3, 2)
    plt.title(f"Plots comparison. Label: {len(label)}")
    plt.ylim(-0.015, 0.015)
    plt.plot(slice)
    for line in lines:
        plt.axvline(line[0], color='r')
        plt.axvline(line[1], color='r')
        plt.axvline(line[2], color='g')

    plt.subplot(1, 3, 3)
    plt.axhline(y=1e-6, color='r')
    plt.ylim(0, 2e-6)
    plt.plot(vars)

    
    plt.show()
    

In [ ]:
indexes = [random.randint(0, 3842) for i in range(100)]
for i in indexes:
    frequencies, times, spectrogram, slice, label, vars, timeSlice = __getitem__(i)
    sliceN = slice-torch.mean(slice)
    multiplot(frequencies, times, spectrogram, sliceN, label, vars, timeSlice)